# **📄 Document type classification baseline code**
> 문서 타입 분류 대회에 오신 여러분 환영합니다! 🎉     
> 아래 baseline에서는 ResNet 모델을 로드하여, 모델을 학습 및 예측 파일 생성하는 프로세스에 대해 알아보겠습니다.

## Contents
- Prepare Environments
- Import Library & Define Functions
- Make Rotate Data

## 1. Prepare Environments

* 데이터 로드를 위한 구글 드라이브를 마운트합니다.
* 필요한 라이브러리를 설치합니다.

## 2. Import Library & Define Functions
* 학습 및 추론에 필요한 라이브러리를 로드합니다.
* 학습 및 추론에 필요한 함수와 클래스를 정의합니다.

In [17]:
# base
import os
import time
import random
import warnings
warnings.filterwarnings('ignore')

# image torch 
import shutil
import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

In [18]:
# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

In [19]:
# 데이터셋 클래스를 정의합니다.
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

## 3. Load Data
* 학습, 테스트 데이터셋과 로더를 정의합니다.

In [21]:
# # Dataset 정의
trn_dataset = ImageDataset(
    "../data/train.csv",
    "../data/train/"
)

# 기존 3500장짜리 서버제출할때 넣는 데이터 
tst_dataset = ImageDataset(
    "../data/sample_submission.csv",
    "../data/test/"
)
print(len(trn_dataset), len(tst_dataset))

1570 3140


In [22]:
# load data
train = pd.read_csv('../data/train.csv')

In [25]:
# 잘못된 레이블 바로잡기 코드 - 귀찮으니 간단하게 노가다 살짝
train['target'][train['ID'] == '45f0d2dfc7e47c03.jpg'] = 7
train['target'][train['ID'] == 'aec62dced7af97cd.jpg'] = 14
train['target'][train['ID'] == '8646f2c3280a4f49.jpg'] = 3
train['target'][train['ID'] == '1ec14a14bbe633db.jpg'] = 7
train['target'][train['ID'] == '7100c5c67aecadc5.jpg'] = 7
train['target'][train['ID'] == 'c5182ab809478f12.jpg'] = 14
train['target'][train['ID'] == '38d1796b6ad99ddd.jpg'] = 10
train['target'][train['ID'] == '0583254a73b48ece.jpg'] = 10

In [26]:
# 저장하기
train.to_csv('../data/train_label_adj.csv', index = False)